# **Imports**

In [245]:
import numpy as np
import pandas as pd
import statistics
import math
import seaborn as sns
import matplotlib.pyplot as plt
import ast
import json
import escher
from escher import Builder
import cobra
from time import sleep

# **File Reading**

In [246]:
simData = np.load(r"../../out/geneRxnVerifData/output.npy", allow_pickle=True, encoding='ASCII')
fluxesWithCaption = simData.tolist()['agents']['0']['listeners']['fba_results']['estimated_fluxes']
complexes = simData.tolist()['agents']['0']['bulk']

ecData = pd.read_csv(r"../new genes/geneData.txt")
ecReactions = ecData["reactionID"].values.tolist()
temp = []
for word in ecReactions:
    if not word == "No Reaction Found":
        temp.append(word)
ecReactions = temp

metabolData = np.load(r"../../out/geneRxnVerifData/stoichiometry.npy", allow_pickle=True, encoding='ASCII')
rxn_metabolites = metabolData.tolist()

# **Collect Reactions Per Metabolite**

In [256]:
rxns = list(rxn_metabolites.keys())
glucoseRxns = {}
fructoseRxns = {}
for rxn in rxns:
    metabolites = rxn_metabolites[rxn].keys()
    for metabolite in metabolites:
        if metabolite == 'PHOSPHO-ENOL-PYRUVATE[c]':
            glucoseRxns[rxn] = rxn_metabolites[rxn]
        if metabolite == 'FRUCTOSE-6P[c]':
            fructoseRxns[rxn] = rxn_metabolites[rxn]

In [ ]:
fructoseRxns

In [257]:
glucoseRxns

{'2.5.1.19-RXN': {'SHIKIMATE-5P[c]': -1,
  'PHOSPHO-ENOL-PYRUVATE[c]': -1,
  '3-ENOLPYRUVYL-SHIKIMATE-5P[c]': 1,
  'Pi[c]': 1},
 '2.5.1.19-RXN (reverse)': {'SHIKIMATE-5P[c]': 1,
  'PHOSPHO-ENOL-PYRUVATE[c]': 1,
  '3-ENOLPYRUVYL-SHIKIMATE-5P[c]': -1,
  'Pi[c]': -1},
 '2.7.1.121-RXN': {'DIHYDROXYACETONE[c]': -1,
  'PHOSPHO-ENOL-PYRUVATE[c]': -1,
  'DIHYDROXY-ACETONE-PHOSPHATE[c]': 1,
  'PYRUVATE[c]': 1},
 '2PGADEHYDRAT-RXN': {'2-PG[c]': -1,
  'PHOSPHO-ENOL-PYRUVATE[c]': 1,
  'WATER[c]': 1},
 '2PGADEHYDRAT-RXN (reverse)': {'2-PG[c]': 1,
  'PHOSPHO-ENOL-PYRUVATE[c]': -1,
  'WATER[c]': -1},
 'KDO-8PSYNTH-RXN': {'CPD-18118[c]': -1,
  'PHOSPHO-ENOL-PYRUVATE[c]': -1,
  'WATER[c]': -1,
  'KDO-8P[c]': 1,
  'Pi[c]': 1},
 'PEPCARBOX-RXN (reverse)': {'OXALACETIC_ACID[c]': 1,
  'Pi[c]': 1,
  'PHOSPHO-ENOL-PYRUVATE[c]': -1,
  'HCO3[c]': -1},
 'PEPCARBOXYKIN-RXN': {'OXALACETIC_ACID[c]': -1,
  'ATP[c]': -1,
  'CARBON-DIOXIDE[c]': 1,
  'PHOSPHO-ENOL-PYRUVATE[c]': 1,
  'ADP[c]': 1},
 'PEPDEPHOS-RXN (reve

# **Find Fluxes for Reactions in Lists**

In [ ]:
fluxes = {}
for rKeys in fructoseRxns.keys():
    fluxes[rKeys] = fluxesWithCaption[rKeys]
fluxes

In [258]:
fluxes2 = {}
for rKeys2 in glucoseRxns.keys():
    fluxes2[rKeys2] = fluxesWithCaption[rKeys2]
fluxes2

{'2.5.1.19-RXN': [58789,
  58789,
  58750,
  58750,
  58747,
  58748,
  58746,
  58746,
  58745,
  58745,
  58707,
  58706,
  58706,
  58706,
  58705,
  58704,
  58704,
  58704,
  58704,
  58704,
  58703,
  58703,
  58703,
  58703,
  58703,
  58702,
  58702,
  58702,
  58702,
  58702],
 '2.5.1.19-RXN (reverse)': [44000,
  43690,
  43760,
  43367,
  43965,
  43469,
  44372,
  43572,
  43636,
  43735,
  43176,
  42857,
  43854,
  43748,
  43632,
  44014,
  43535,
  43327,
  43175,
  43956,
  43178,
  43782,
  44016,
  43430,
  42952,
  43513,
  43980,
  43752,
  43092,
  43558],
 '2.7.1.121-RXN': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 '2PGADEHYDRAT-RXN': [313715,
  331683,
  350947,
  351384,
  377206,
  381922,
  395907,
  408164,
  427026,
  423068,
  446925,
  454875,
  452584,
  462057,
  472240,
  466196,
  486011,
  491198,
  498646,
  488751,
  503154,
  495810,
  492659

# **Establish Glycolysis**

In [221]:
glycolysis = ['TRANS-RXN-157-PTSH-PHOSPHORYLATED','PGLUCISOM-RXN', '6PFRUCTPHOS-RXN', 'F16BDEPHOS-RXN', 'F16ALDOLASE-RXN', 'TRIOSEPISOMERIZATION-RXN', 'GAPOXNPHOSPHN-RXN', 'PHOSGLYPHOS-RXN', 'RXN-15513', '3PGAREARR-RXN', '2PGADEHYDRAT-RXN', 'PEPDEPHOS-RXN', 'PEPSYNTH-RXN']

In [222]:
glycolysisFluxes = {}
foundNames = []
for rxn in glycolysis:
    for key in fluxesWithCaption:
        if rxn in key:
            foundNames.append((rxn, key))

In [223]:
foundNames

[('TRANS-RXN-157-PTSH-PHOSPHORYLATED',
  'TRANS-RXN-157-PTSH-PHOSPHORYLATED/ALPHA-GLUCOSE//ALPHA-GLC-6-P/PTSH-MONOMER.62.'),
 ('TRANS-RXN-157-PTSH-PHOSPHORYLATED',
  'TRANS-RXN-157-PTSH-PHOSPHORYLATED/ALPHA-GLUCOSE//D-glucopyranose-6-phosphate/PTSH-MONOMER.76.'),
 ('TRANS-RXN-157-PTSH-PHOSPHORYLATED',
  'TRANS-RXN-157-PTSH-PHOSPHORYLATED/ALPHA-GLUCOSE//GLC-6-P/PTSH-MONOMER.56.'),
 ('TRANS-RXN-157-PTSH-PHOSPHORYLATED',
  'TRANS-RXN-157-PTSH-PHOSPHORYLATED/GLC//ALPHA-GLC-6-P/PTSH-MONOMER.52.'),
 ('TRANS-RXN-157-PTSH-PHOSPHORYLATED',
  'TRANS-RXN-157-PTSH-PHOSPHORYLATED/GLC//D-glucopyranose-6-phosphate/PTSH-MONOMER.66.'),
 ('TRANS-RXN-157-PTSH-PHOSPHORYLATED',
  'TRANS-RXN-157-PTSH-PHOSPHORYLATED/GLC//GLC-6-P/PTSH-MONOMER.46.'),
 ('TRANS-RXN-157-PTSH-PHOSPHORYLATED',
  'TRANS-RXN-157-PTSH-PHOSPHORYLATED/Glucopyranose//ALPHA-GLC-6-P/PTSH-MONOMER.62.'),
 ('TRANS-RXN-157-PTSH-PHOSPHORYLATED',
  'TRANS-RXN-157-PTSH-PHOSPHORYLATED/Glucopyranose//D-glucopyranose-6-phosphate/PTSH-MONOMER.76.'),


In [224]:
prev = ""
for names in foundNames:
    if prev == names[0]:
        for i in range(len(glycolysisFluxes[prev])):
            if "(reverse)" in names[1]:
                glycolysisFluxes[prev][i] -= fluxesWithCaption[names[1]][i]
            else:
                glycolysisFluxes[prev][i] += fluxesWithCaption[names[1]][i]
    else:
        prev = names[0]
        glycolysisFluxes[prev] = fluxesWithCaption[names[1]]

In [225]:
glycolysisFluxes

{'TRANS-RXN-157-PTSH-PHOSPHORYLATED': [119867,
  134937,
  131366,
  135425,
  157254,
  162582,
  170203,
  188835,
  202432,
  191708,
  204456,
  211206,
  219579,
  219709,
  236177,
  226953,
  237373,
  240431,
  252837,
  239061,
  253680,
  255057,
  248533,
  251083,
  266146,
  260177,
  263802,
  258229,
  265732,
  258062],
 'PGLUCISOM-RXN': [111229,
  133898,
  119677,
  123328,
  134490,
  140246,
  141530,
  157985,
  165593,
  156975,
  166363,
  171833,
  174825,
  174491,
  185906,
  178597,
  186471,
  188897,
  198320,
  185400,
  197317,
  196820,
  191268,
  194016,
  206602,
  200324,
  201853,
  198121,
  204968,
  197603],
 '6PFRUCTPHOS-RXN': [81988,
  81988,
  81988,
  81988,
  81988,
  81988,
  81988,
  81988,
  81988,
  81988,
  81988,
  81988,
  81988,
  81988,
  81988,
  81988,
  81987,
  81987,
  81987,
  81987,
  81987,
  81987,
  81987,
  81987,
  81987,
  81987,
  81987,
  81987,
  81987,
  81987],
 'F16BDEPHOS-RXN': [5201,
  5201,
  5215,
  5215,
  52

In [226]:
fluxLast = {}

for key in glycolysisFluxes.keys():
    fluxes = glycolysisFluxes[key]
    fluxLast[key] = fluxes[-1]

In [227]:
fluxLast

{'TRANS-RXN-157-PTSH-PHOSPHORYLATED': 258062,
 'PGLUCISOM-RXN': 197603,
 '6PFRUCTPHOS-RXN': 81987,
 'F16BDEPHOS-RXN': 5253,
 'F16ALDOLASE-RXN': 75768,
 'TRIOSEPISOMERIZATION-RXN': -245635,
 'GAPOXNPHOSPHN-RXN': 517023,
 'PHOSGLYPHOS-RXN': -517021,
 'RXN-15513': 0,
 '3PGAREARR-RXN': -516254,
 '2PGADEHYDRAT-RXN': 516362,
 'PEPDEPHOS-RXN': 0,
 'PEPSYNTH-RXN': 0}

In [228]:
escher.list_available_maps()

[{'organism': 'Saccharomyces cerevisiae',
  'map_name': 'iMM904.Central carbon metabolism'},
 {'organism': 'Homo sapiens',
  'map_name': 'RECON1.Inositol retinol metabolism'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Glycolysis TCA PPP'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Tryptophan metabolism'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Carbohydrate metabolism'},
 {'organism': 'Homo sapiens',
  'map_name': 'RECON1.Amino acid metabolism (partial)'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Nucleotide metabolism'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Fatty acid biosynthesis (saturated)'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Nucleotide and histidine biosynthesis'},
 {'organism': 'Escherichia coli', 'map_name': 'e_coli_core.Core metabolism'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Central metabolism'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Fatty acid beta-oxidation'}

In [229]:
escher.list_available_models()

[{'organism': 'Saccharomyces cerevisiae', 'model_name': 'iMM904'},
 {'organism': 'Homo sapiens', 'model_name': 'RECON1'},
 {'organism': 'Escherichia coli', 'model_name': 'e_coli_core'},
 {'organism': 'Escherichia coli', 'model_name': 'iJO1366'}]

In [230]:
with open('../../iJO1366.Central metabolism.json') as json_file:
    data = json.load(json_file)

In [231]:
escherRxns = data[1]['reactions']
names = []

for number in escherRxns:
    reactionName = escherRxns[number]['name']
    names.append(reactionName)

In [232]:
enzymeNames = ['glucose-6-phosphate isomerase', '6-phosphofructokinase', 'fructose-bisphosphatase', 'fructose-bisphosphate aldolase', 'triose-phosphate isomerase', 'glyceraldehyde-3-phosphate dehydrogenase (phosphorylating)', 'phosphoglycerate kinase', 'pyruvate kinase', 'phosphoglycerate mutase (2,3-diphosphoglycerate-dependent)', 'pyruvate-water dikinase (phosphorylating)', 'phosphopyruvate hydratase']

In [233]:
glycolysis 

['TRANS-RXN-157-PTSH-PHOSPHORYLATED',
 'PGLUCISOM-RXN',
 '6PFRUCTPHOS-RXN',
 'F16BDEPHOS-RXN',
 'F16ALDOLASE-RXN',
 'TRIOSEPISOMERIZATION-RXN',
 'GAPOXNPHOSPHN-RXN',
 'PHOSGLYPHOS-RXN',
 'RXN-15513',
 '3PGAREARR-RXN',
 '2PGADEHYDRAT-RXN',
 'PEPDEPHOS-RXN',
 'PEPSYNTH-RXN']

In [234]:
want = []
for rxn in enzymeNames:
    for name in names:
        if name.lower() in rxn.lower() or rxn.lower() in name.lower():
            want.append((rxn, name))

In [235]:
nameAssociation = {
    'TRANS-RXN-157-PTSH-PHOSPHORYLATED': 'D-glucose transport via PEP:Pyr PTS',
    'PGLUCISOM-RXN': 'Glucose-6-phosphate isomerase',
    '6PFRUCTPHOS-RXN': 'Phosphofructokinase',
    'F16BDEPHOS-RXN': 'Fructose-bisphosphatase',
    'F16ALDOLASE-RXN': 'Fructose-bisphosphate aldolase',
    'TRIOSEPISOMERIZATION-RXN': 'Triose-phosphate isomerase',
    'GAPOXNPHOSPHN-RXN': 'Glyceraldehyde-3-phosphate dehydrogenase',
    'PHOSGLYPHOS-RXN': 'Phosphoglycerate kinase',
    'RXN-15513': 'Phosphoglycerate mutase',
    '3PGAREARR-RXN': 'Phosphoglycerate mutase',
    '2PGADEHYDRAT-RXN': 'Enolase',
    'PEPDEPHOS-RXN': 'Pyruvate kinase',
    'PEPSYNTH-RXN': 'Phosphoenolpyruvate synthase'
}

In [236]:
escherKeys = []
currentKeys = glycolysisFluxes.keys()

for key in currentKeys:
    new = nameAssociation[key]
    escherKeys.append(new)

In [237]:
escherKeys

['D-glucose transport via PEP:Pyr PTS',
 'Glucose-6-phosphate isomerase',
 'Phosphofructokinase',
 'Fructose-bisphosphatase',
 'Fructose-bisphosphate aldolase',
 'Triose-phosphate isomerase',
 'Glyceraldehyde-3-phosphate dehydrogenase',
 'Phosphoglycerate kinase',
 'Phosphoglycerate mutase',
 'Phosphoglycerate mutase',
 'Enolase',
 'Pyruvate kinase',
 'Phosphoenolpyruvate synthase']

In [238]:
fluxLast

{'TRANS-RXN-157-PTSH-PHOSPHORYLATED': 258062,
 'PGLUCISOM-RXN': 197603,
 '6PFRUCTPHOS-RXN': 81987,
 'F16BDEPHOS-RXN': 5253,
 'F16ALDOLASE-RXN': 75768,
 'TRIOSEPISOMERIZATION-RXN': -245635,
 'GAPOXNPHOSPHN-RXN': 517023,
 'PHOSGLYPHOS-RXN': -517021,
 'RXN-15513': 0,
 '3PGAREARR-RXN': -516254,
 '2PGADEHYDRAT-RXN': 516362,
 'PEPDEPHOS-RXN': 0,
 'PEPSYNTH-RXN': 0}

In [239]:
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='e_coli_core',
    reaction_data={r: v for r, v in zip(escherKeys, fluxLast.values())},
)

In [240]:
builder

Builder(reaction_data={'D-glucose transport via PEP:Pyr PTS': 258062, 'Glucose-6-phosphate isomerase': 197603,…

In [ ]:
{'2-PG[c]': -1, 'G3P[c]': 1}